### Yelp API Calls-Revised
#### Laura Dimaria

7/30/22

In [52]:
#Imports
import numpy as np
import pandas as pd

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [53]:
#load yelp api file
with open('/Users/lauradimaria/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [54]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [55]:
#set location and term
LOCATION = 'Philadelphia, PA, 19125'
TERM = 'Mexican'

In [56]:
#create json file
JSON_FILE = f"Data/results_in_progress_mexican.json"
JSON_FILE

'Data/results_in_progress_mexican.json'

In [57]:
#check if json file exists
file_exists = os.path.isfile(JSON_FILE)
 
if file_exists == False:
    folder = os.path.dirname(JSON_FILE)

    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
        
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  

else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_mexican.json already exists.


In [58]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 257 previous results found.


In [59]:
#get results using api search
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [60]:
#total number of results
total_results = results['total']
total_results

257

In [61]:
#number of results per page
results_per_page = len(results['businesses'])
results_per_page

0

In [62]:
#import time and math packages
import time, math
#figure out how many pages of results we have
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

ZeroDivisionError: division by zero

In [63]:
#join with previous results
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [64]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/13 [00:00<?, ?it/s]

In [65]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)


  0%|          | 0/13 [00:00<?, ?it/s]

In [66]:
final_mexican_df = pd.read_json(JSON_FILE)
display(final_mexican_df.head(), final_mexican_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,GAuweBrcC_wK3a0UhZjwfA,loco-pez-philadelphia-3,Loco Pez,https://s3-media3.fl.yelpcdn.com/bphoto/LVTiVH...,False,https://www.yelp.com/biz/loco-pez-philadelphia...,664,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.977142, 'longitude': -75.127202}","[delivery, pickup]",$$,"{'address1': '2401 E Norris St', 'address2': '...",+12678868061,(267) 886-8061,274.985962
1,9UOYOZHb4-V3hZlPOxUkVQ,cantina-la-martina-philadelphia,Cantina La Martina,https://s3-media4.fl.yelpcdn.com/bphoto/0wT_bs...,False,https://www.yelp.com/biz/cantina-la-martina-ph...,19,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 39.99157, 'longitude': -75.1228177}","[delivery, pickup]",NaN,"{'address1': '2800 D St', 'address2': None, 'a...",+12675192142,(267) 519-2142,1639.405365
2,asbEI02GRGFPSPM97hrIaA,nemi-restaurant-philadelphia-2,Nemi Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/5YF91d...,False,https://www.yelp.com/biz/nemi-restaurant-phila...,127,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 39.9816738971912, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '2636 E Ann St', 'address2': 'Flr...",+12675190713,(267) 519-0713,1383.239225
3,zvvl3c1FO3O3BZdhusficA,lmno-philadelphia,LMNO,https://s3-media3.fl.yelpcdn.com/bphoto/i5R9Ri...,False,https://www.yelp.com/biz/lmno-philadelphia?adj...,87,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 39.97604, 'longitude': -75.13377}",[],NaN,"{'address1': '1739-49 N Front St', 'address2':...",+12157707001,(215) 770-7001,852.382979
4,5ItgryJvadUrKVljjJ8l4g,que-chula-es-puebla-philadelphia-2,Que Chula Es Puebla,https://s3-media2.fl.yelpcdn.com/bphoto/JQ4LfL...,False,https://www.yelp.com/biz/que-chula-es-puebla-p...,263,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 39.9723663330078, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1356 N 2nd St', 'address2': '', ...",+12152030404,(215) 203-0404,1387.901484


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
252,ErJ9NtMXt-WJZV_TVZ08Ag,green-garden-philadelphia-2,Green Garden,https://s3-media2.fl.yelpcdn.com/bphoto/HTH4pS...,False,https://www.yelp.com/biz/green-garden-philadel...,82,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 39.94718, 'longitude': -75.15746}","[pickup, delivery]",$,"{'address1': '237 S 10th St', 'address2': '', ...",+12153511118,(215) 351-1118,4358.373399
253,rnecTemi_wnY5fjBng77Kw,kayu-philadelphia,Kayu,https://s3-media3.fl.yelpcdn.com/bphoto/h2qmnF...,False,https://www.yelp.com/biz/kayu-philadelphia?adj...,30,"[{'alias': 'salad', 'title': 'Salad'}, {'alias...",3.5,"{'latitude': 39.9554942816928, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '1717 Arch St', 'address2': '', '...",+12676399949,(267) 639-9949,4506.149814
254,E1gf1YIWOo1BgzMUwJtEZg,top-tomato-bar-and-pizza-philadelphia,Top Tomato Bar & Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/2ImEYd...,False,https://www.yelp.com/biz/top-tomato-bar-and-pi...,250,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 39.94958, 'longitude': -75.15897}","[pickup, delivery]",$,"{'address1': '116 S 11th St', 'address2': '', ...",+12155928242,(215) 592-8242,4252.640408
255,n-iPn34tLaqnq-dI8m8Bkw,howl-at-the-moon-philadelphia-philadelphia,Howl At the Moon Philadelphia,https://s3-media1.fl.yelpcdn.com/bphoto/7ArkAn...,False,https://www.yelp.com/biz/howl-at-the-moon-phil...,229,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.0,"{'latitude': 39.9477765, 'longitude': -75.1664...",[],$$,"{'address1': '258 S 15th St', 'address2': '', ...",+12675192189,(267) 519-2189,4850.734991
256,wPi7p5fZyGTOjDU28IlTqA,woodys-philadelphia,Woody's,https://s3-media2.fl.yelpcdn.com/bphoto/mv0mb4...,False,https://www.yelp.com/biz/woodys-philadelphia?a...,340,"[{'alias': 'danceclubs', 'title': 'Dance Clubs...",3.0,"{'latitude': 39.949064, 'longitude': -75.16241}",[],$$,"{'address1': '202 S 13th St', 'address2': '', ...",+12155451893,(215) 545-1893,4501.862180


In [67]:
final_mexican_df.duplicated(subset='id').sum()

0

In [68]:
final_mexican_df.to_csv('Data/final_results_mexican.csv.gz', compression='gzip',index=False)

In [69]:
final_mexican_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             257 non-null    object 
 1   alias          257 non-null    object 
 2   name           257 non-null    object 
 3   image_url      257 non-null    object 
 4   is_closed      257 non-null    bool   
 5   url            257 non-null    object 
 6   review_count   257 non-null    int64  
 7   categories     257 non-null    object 
 8   rating         257 non-null    float64
 9   coordinates    257 non-null    object 
 10  transactions   257 non-null    object 
 11  price          208 non-null    object 
 12  location       257 non-null    object 
 13  phone          257 non-null    object 
 14  display_phone  257 non-null    object 
 15  distance       257 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 30.5+ KB


#### I couldn't figure out what was wrong with my french DF I had originally had, but I need to get these assignments completed so I just redid the assignment with a different cuisine.